# Symbolic Fuzzing

One of the problems with traditional methods of fuzzing is that they fail to penetrate deeply into the program. Quite often the execution of a specific branch of execution may happen only with very specific inputs, which may represent an extremely small fraction of the input space. The traditional fuzzing methods relies on chance to produce inputs they need. However, relying on randomness to generate values that we want is a bad idea when the space to be explored is large. For example, given a function that accepts a string, even if one only considers the first $10$ characters, already has $2^{80}$ possible inputs. If one is looking for a specific string, random generation of values will take a few thousand years even in one of the super computers.

Symbolic execution is a way out of this problem. A program is a computation that can be treated as a system of equations that obtains the output values from the given inputs. Executing the program symbolically -- that is, solving these mathematically -- along with any specified objective such as covering a particular branch or obtaining a particular output will get us inputs that can accomplish this task. In this chapter, we investigate how _symbolic execution_ can be implemented, and how it can be used to obtain interesting values for fuzzing.

**Prerequisites**

* You should have read the [chapter on coverage](Coverage.ipynb).
* Some knowledge of inheritance in Python is required.
* A familiarity with the [chapter on search based fuzzing](SearchBasedFuzzer.ipynb) would be useful.

## Using Symbolic Variables for Coverage

In the chapter on [parsing and recombining inputs](SearchBasedFuzzer.ipynb), we saw how difficult it was to generate inputs for `process_vehicle()` -- a simple function that accepts a string. The solution given there was to rely on preexisting sample inputs. However, this solution is inadequate as it assumes the existence of sample inputs. What if there are sample inputs at hand?

For a simpler example, let us consider the following function. Can we generate inputs to cover all the paths?

In [ ]:
def check_triangle(a,b,c):
    if a == b:
        if a == c:
            if b == c:
                return "Equilateral"
            else:
                return "Isosceles"
        else:
            return "Isosceles"
    else:
        if b != c:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"
        else:
              return "Isosceles"

The control flow graph of this function can be represented as follows:

In [ ]:
from graphviz import Source, Graph

In [ ]:
import fuzzingbook_utils

In [ ]:
from ControlFlow import PyCFG, CFGNode, to_graph, gen_cfg

In [ ]:
import inspect

In [ ]:
Source(to_graph(gen_cfg(inspect.getsource(check_triangle))))

The possible execution paths traced by the program can be represented as follows.

The function takes three parameters, and the possible execution paths are the following.

```python
1: [1, 2, 3, 4, 5, Equilateral]
2: [1, 2, 3, 4, 7, Isosceles]
3: [1, 2, 3, 9, Isosceles]
4: [1, 2, 11, 12, 13, Isosceles]
5: [1, 2, 11, 12, 15, Scalene]
6: [1, 2, 11, 17, Isosceles]
```

If we want to cover the path <1>, we need to solve the following constraints.

In [ ]:
import z3

In [ ]:
a, b, c = z3.Ints('a b c')

In [ ]:
z3.solve(a == b, a == c, b == c)

In [ ]:
assert check_triangle(0, 0, 0) == 'Equilateral'

Similarly, for solving path <2> we need to simply invert the condition (2):

In [ ]:
a, b, c = z3.Ints('a b c')

In [ ]:
z3.solve(a == b, a == c, z3.Not(b == c))

The symbolic execution suggests that there is no solution. A moment's reflection will convince us that it is indeed true. Let us proceed with the other paths.

Next we attempt path <3> which we get by inverting (4)

In [ ]:
a, b, c = z3.Ints('a b c')

In [ ]:
z3.solve(a == b, z3.Not(a==c))

In [ ]:
assert check_triangle(1, 1, 0) == 'Isosceles'

How about path <4>?

In [ ]:
a, b, c = z3.Ints('a b c')

In [ ]:
z3.solve(z3.Not(a == b), b!= c, a == c)

In [ ]:
assert check_triangle(1, 0, 1) == 'Isosceles'

Continuing to path <5>:

In [ ]:
a, b, c = z3.Ints('a b c')

In [ ]:
z3.solve(z3.Not(a == b), b!= c, z3.Not(a == c))

This is surprising! We get negative numbers, because while we *know* that a triangle's sides should not have negative numbers, it was not included in the code. We can explore how it would look if that restriction was added.

In [ ]:
z3.solve(a >=0, b>=0, c>= 0,z3.Not(a == b), b!= c, z3.Not(a == c))

And indeed it is a *Scalene* triangle.

In [ ]:
assert check_triangle(1, 0, 2) == 'Scalene'

For path <6> the procedure is smimilar.

In [ ]:
z3.solve(a >=0, b>=0, c>= 0,z3.Not(a == b), z3.Not(b!= c))

In [ ]:
assert check_triangle(0, 1, 1) == 'Isosceles'

That is, using simple symbolic computation, we were able to easily see that (1) some of the paths are not reachable, and (2) some of the conditions were insufficient. What about coverage?

In [ ]:
from Coverage import Coverage

In [ ]:
with Coverage() as cov:
    assert check_triangle(0, 0, 0) == 'Equilateral'
    assert check_triangle(1, 1, 0) == 'Isosceles'
    assert check_triangle(1, 0, 1) == 'Isosceles'
    assert check_triangle(1, 0, 2) == 'Scalene'
    assert check_triangle(0, 1, 1) == 'Isosceles'

In [ ]:
source = inspect.getsource(check_triangle).strip().split('\n')

In [ ]:
covered = set([lineno for method,lineno in cov._trace])
for i,s in enumerate(source):
    print('%s %2d: %s' % ('#' if i+1 in covered else ' ', i+1, s))

The coverage is as expected. The generated values does seem to cover all code that can be covered. However, doing this by hand is tedious and error prone. What we need is the ability to extract *all paths* in the program, and symbolically execute each path, which will generate the inputs required to cover all reachable portions of the program.

Indeed, doing this is fairly simple for a simple program such as `check_triangle()`. We first define `get_all_paths()` that, given a starting point, will recursively examine all child nodes, and return the traversed paths.

### Simple Symbolic Fuzzer

In [ ]:
def get_all_paths(fenter):
    if not fenter.children:
        yield [(0, fenter)]
        
    for idx, child in enumerate(fenter.children):
        for path in get_all_paths(child):
            yield [(idx, fenter)] + path

In [ ]:
def extract_conditions(path):
    last = None
    conditions = []
    for (idx, elt) in path:
        if last is not None:
            j = last.to_json()
            l = last.child_node_annotations.get(elt.rid)
            if j['ast'].startswith('_if:'):
                conditions.append(("%s" if l == 'T' else "z3.Not%s") % j['ast'][5:])
        last = elt
    return conditions

In [ ]:
cfg = PyCFG()
cfg.gen_cfg(inspect.getsource(check_triangle))
fnenter, fnexit = cfg.functions['check_triangle']

my_conditions = []
for path in get_all_paths(fnenter):
    conditions = extract_conditions(path)
    s = 'z3.solve(%s)' % ', '.join(conditions)
    my_conditions.append(s)
    print(s)

In [ ]:
for s in my_conditions:
    print(s)
    a, b, c = z3.Ints('a b c')
    eval(s)
    print()

#### Problems

* Reassignments
* Loops

In [ ]:
def gcd(a, b):
    if a<b:
        c = a
        a = b
        b = c

    while b != 0 :
        c = a
        a = b
        b = c % b
    return a

In [ ]:
Source(to_graph(gen_cfg(inspect.getsource(gcd))))

In [ ]:
from ExpectError import ExpectError

In [ ]:
cfg = PyCFG()
cfg.gen_cfg(inspect.getsource(gcd))
fnenter, fnexit = cfg.functions['gcd']
with ExpectError():
    my_conditions = []
    for path in get_all_paths(fnenter):
        conditions = extract_conditions(path)
        s = 'z3.solve(%s)' % ', '.join(conditions)
        my_conditions.append(s)
        print(s)

### Advanced Symbolic Fuzzer

In [ ]:
import ast
import astunparse

#### names

The method `names()` retrieves the variable names used in an expression.

In [ ]:
def names(astnode):
    lst = []
    if isinstance(astnode, ast.BoolOp):
        for i in astnode.values:
            lst.extend(names(i))
    elif isinstance(astnode, ast.BinOp):
        lst.extend(names(astnode.left))
        lst.extend(names(astnode.right))
    elif isinstance(astnode, ast.UnaryOp):
        lst.extend(names(astnode.operand))
    elif isinstance(astnode, ast.Call):
        for i in astnode.args:
            lst.extend(names(i))
    elif isinstance(astnode, ast.Compare):
        lst.extend(names(astnode.left))
        for i in astnode.comparators:
            lst.extend(names(i))
    elif isinstance(astnode, ast.Name):
        lst.append(astnode.id)
    elif isinstance(astnode, ast.Expr):
        lst.extend(names(astnode.value))
    elif isinstance(astnode, (ast.Num, ast.Str, ast.Tuple)):
        pass
    else:
        raise Exception(str(astnode))
    return lst

In [ ]:
def get_expression(src):
    return ast.parse(src).body[0].value

In [ ]:
v = get_expression('fn(x+z,y>(a+b)) == c')
names(v)

#### ssa_transform

Use the current `env` to determine what to rename variables to. if `env[v] == 1`, `v` is renamed to `_v_1`

In [ ]:
def ssa_transform(astnode, env):
    if isinstance(astnode, ast.BoolOp):
        fn = 'z3.And' if isinstance(astnode.op, ast.And) else 'z3.Or'
        return ast.Call(
            ast.Name(fn, None),
            [ssa_transform(i, env) for i in astnode.values], [])
    elif isinstance(astnode, ast.BinOp):
        return ast.BinOp(
            ssa_transform(astnode.left, env), astnode.op,
            ssa_transform(astnode.right, env))
    elif isinstance(astnode, ast.UnaryOp):
        if isinstance(astnode.op, ast.Not):
            return ast.Call(
                ast.Name('z3.Not', None),
                [ssa_transform(astnode.operand, env)], [])
        else:
            return ast.UnaryOp(astnode.op, ssa_transform(astnode.operand, env))
    elif type(astnode) is ast.Call:
        return ast.Call(astnode.func,
                        [ssa_transform(i, env) for i in astnode.args],
                        astnode.keywords)
    elif type(astnode) is ast.Compare:
        return ast.Compare(
            ssa_transform(astnode.left, env), astnode.ops,
            [ssa_transform(i, env) for i in astnode.comparators])
    elif type(astnode) is ast.Name:
        if astnode.id not in env:
            env[astnode.id] = 0
        num = env[astnode.id]
        return ast.Name('_%s_%d' % (astnode.id, num), astnode.ctx)
    elif type(astnode) is ast.Return:
        return ast.Return(ssa_transform(astnode.value, env))
    else:
        return astnode

In [ ]:
def to_src(astnode):
    return astunparse.unparse(astnode).strip()

In [ ]:
env = {}

In [ ]:
ba = get_expression('x == 1 and y == 2')
type(ba)

In [ ]:
assert to_src(ssa_transform(ba, env)) == 'z3.And((_x_0 == 1), (_y_0 == 2))'

In [ ]:
bo = get_expression('x == 1 or y == 2')
type(bo.op)

In [ ]:
assert to_src(ssa_transform(bo, env)) == 'z3.Or((_x_0 == 1), (_y_0 == 2))'

In [ ]:
b = get_expression('x + y')
type(b)

In [ ]:
assert to_src(ssa_transform(b, env)) == '(_x_0 + _y_0)'

In [ ]:
u = get_expression('-y')
type(u)

In [ ]:
assert to_src(ssa_transform(u, env)) == '(- _y_0)'

In [ ]:
un = get_expression('not y')
type(un.op)

In [ ]:
assert to_src(ssa_transform(un, env)) == 'z3.Not(_y_0)'

In [ ]:
c = get_expression('x == y')
type(c)

In [ ]:
assert to_src(ssa_transform(c, env)) == '(_x_0 == _y_0)'

In [ ]:
f = get_expression('fn(x,y)')
type(f)

In [ ]:
assert to_src(ssa_transform(f, env)) == 'fn(_x_0, _y_0)'

In [ ]:
env

#### PNode

For keeping track of assignments, we need a bit more infrastructure.

In [ ]:
class PNode:
    def __init__(self, idx, cfgnode, parent=None, order=0):
        self.idx, self.cfgnode, self.parent, self.order = idx, cfgnode, parent, order
        
    def __repr__(self):
        return "PNode:%d[%s order:%d]" % (self.idx, str(self.cfgnode), self.order)

Using it.

In [ ]:
cfg = PyCFG()
cfg.gen_cfg(inspect.getsource(gcd))
fnenter, fnexit = cfg.functions['gcd']

In [ ]:
PNode(0, fnenter)

##### copy
The `copy()` method generates a copy for the child's keep, indicating which path was taken (with `order` of the child).

In [ ]:
class PNode(PNode):
    def copy(self, order):
        return PNode(self.idx, self.cfgnode, self.parent, order)

Using it.

In [ ]:
PNode(0, fnenter).copy(1)

##### explore

Explore the children if any (one step). If done exhaustively, this will generate all paths from a starting node until no more children are left. We made `PNode` to a container class so that this iteration can be driven from outside, and stopped if say a maximum iteration is complete, or certain paths need to be prioritized.

In [ ]:
class PNode(PNode):
    def explore(self):
        return [
            PNode(self.idx + 1, n, self.copy(i))
            for (i, n) in enumerate(self.cfgnode.children)
        ]

Using it.

In [ ]:
PNode(0, fnenter).explore()

In [ ]:
PNode(0, fnenter).explore()[0].explore()

##### get_path_to_root

The method `get_path_to_root()` recursively goes up through child->parent chain retrieving the complete chain to the topmost parent.

In [ ]:
class PNode(PNode):
    def get_path_to_root(self):
        path = []
        n = self
        while n:
            path.append(n)
            n = n.parent
        return list(reversed(path))

In [ ]:
p = PNode(0, fnenter)
[s.get_path_to_root() for s in p.explore()[0].explore()[0].explore()[0].explore()]

The string representation of the node is in `z3` solvable form.

In [ ]:
class PNode(PNode):
    def __str__(self):
        path = self.get_path_to_root()
        #print([p.cfgnode.lineno() for p in path])
        ssa_path = to_single_assignment(path)
        return ', '.join([to_src(p) for p in ssa_path])

However, before using it, we need to define the `to_single_assignment()`. But first, we define `names()`.

###### to_single_assignment

Rename used variables. Any variable `v = xxx` becomes renamed to `_v_0` and any later assignment such as `v = v + 1` is transformed to `_v_1 = _v_0 + 1` and later conditionals such as `v == x` gets transformed to `(_v_1 == _x_0)`.

In [ ]:
def to_single_assignment(path):
    env = {}
    my_vars = set()
    new_path = []
    for node in path:
        ast_node = node.cfgnode.ast_node
        new_node = None
        if type(ast_node) is ast.AnnAssign and ast_node.target.id in {'enter', 'exit'}:
            continue
        elif type(ast_node) is ast.AnnAssign and ast_node.target.id in {'_if', '_while'}:
            new_node = ssa_transform(ast_node.annotation, env)
            if node.order != 0:
                assert node.order == 1
                new_node = ast.Call(ast.Name('z3.Not', None), [new_node], [])
        elif type(ast_node) is ast.Assign:
            assigned = ast_node.targets[0].id
            val = [ssa_transform(ast_node.value, env)]
            if assigned not in env:
                env[assigned] = 0
            else:
                env[assigned] += 1
            target = ast.Name('_%s_%d' % (ast_node.targets[0].id, env[assigned]), None)
            new_node = ast.Expr(ast.Compare(target, [ast.Eq()], val))
        elif type(ast_node) is ast.Return:
             continue
        elif type(ast_node) is ast.Pass:
             continue
        else:
            raise Exception('NI')
        new_path.append(new_node)
    return new_path

In [ ]:
p = PNode(0, fnenter)
path = p.explore()[0].explore()[0].explore()[0].get_path_to_root()
spath = to_single_assignment(path)

In [ ]:
[to_src(p) for p in spath]

In [ ]:
assert set(q for p in spath for q in names(p)) == {'_a_0', '_a_1', '_b_0', '_c_0'}

##### can_be_satisfied

In [ ]:
def to_z3_eqn(spath):
    spath = to_single_assignment(path)
    my_names = set(q for p in spath for q in names(p))
    s1 = "%s = z3.Ints('%s')" % (', '.join(my_names), ' '.join(my_names))
    s2 = ', '.join([to_src(i) for i in spath])
    return s1, s2

In [ ]:
class PNode(PNode):
    def can_be_satisfied(self):
        s1, s2 = to_z3_eqn(self.get_path_to_root())
        s = z3.Solver()
        exec(s1, globals(), locals())
        exec("s.add(%s)" % s2, globals(), locals())
        if s.check() == z3.sat:
            self.mode = s.model()
            return True
        else:
            return False

#### get_all_paths

Get all paths one can generate from function enter node (`fenter`) subject to max_iter limit.

In [ ]:
def get_all_paths(fenter, max_iter):
    path_lst = [PNode(0, fenter)]
    completed = []

    for i in range(0, max_iter):
        new_paths = []
        for path in path_lst:
            # explore each path once
            if path.cfgnode.children:
                np = path.explore()
                for p in np:
                    if p.can_be_satisfied():
                        new_paths.append(p)
                    else:
                        pass
            else:
                completed.append(path)
        path_lst = new_paths
    return completed + path_lst

In [ ]:
cfg = PyCFG()
cfg.gen_cfg(inspect.getsource(gcd))
fnenter, fnexit = cfg.functions['gcd']
nodes = get_all_paths(fnenter, 10)
data = []
for node in nodes:
    path = node.get_path_to_root()
    s1, s2 = to_z3_eqn(spath)
    exec(s1, globals(), locals())
    s = z3.Solver()
    s2 = 's.add(%s)' % s2
    print(s2)
    exec(s2, globals(), locals())
    if s.check() == z3.sat:
        m = s.model()
        data.append((m.eval(_a_0).as_long(), m.eval(_b_0).as_long()))
    print()

In [ ]:
with Coverage() as cov:
    for a,b in data:
        gcd(a,b)

In [ ]:
covered = set([lineno for method,lineno in cov._trace])
source = inspect.getsource(gcd).strip().split('\n')
for i,s in enumerate(source):
    print('%s %2d: %s' % ('#' if i+1 in covered else ' ', i+1, s))

## Symbolic Execution

In [ ]:
import PyExZ3.pyloader

In [ ]:
gi, rv, path = PyExZ3.pyloader.exploreFunction(check_triangle)

In [ ]:
Source(path.toDot())

## Lessons Learned

* One can use symbolic execution to augment the inputs that explore all characteristics of a program.

## Next Steps

_Link to subsequent chapters (notebooks) here:_

## Background

\cite{KLEE}

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_